In [3]:
import pandas as pd
import glob
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Path to the folder containing the CSV files
path = "vital_data/"

# Read the ICU days dataframe
case_df = pd.read_csv("https://api.vitaldb.net/cases")  # Adjust the filename as necessary

In [4]:


# # take random 250 cases
# icu_days_df = case_df.sample(n=250, random_state=42)

data_icu_greater1 = case_df[case_df["icu_days"] > 1]
# take random 40 caseids
caseids = data_icu_greater1["caseid"].unique()
np.random.seed(42)
np.random.shuffle(caseids)
caseids = caseids[:200]
data_icu_greater1 = data_icu_greater1[data_icu_greater1["caseid"].isin(caseids)]

# only take data with icu_days <= 1
data_icu_smaller1 = case_df[case_df["icu_days"] <= 1]
# take random 40 caseids
caseids = data_icu_smaller1["caseid"].unique()
np.random.seed(42)
np.random.shuffle(caseids)
caseids = caseids[:200]
data_icu_smaller1 = data_icu_smaller1[data_icu_smaller1["caseid"].isin(caseids)]


# Combine the two datasets
icu_days_df = pd.concat([data_icu_greater1, data_icu_smaller1])

In [5]:
# Read all case CSV files
all_files = glob.glob(path + "case_*.csv")

# Create a dictionary to hold the dataframes
case_data = {}
for filename in all_files:
    caseid = int(filename.split("_")[-1].split(".")[0])  # Extract caseid from filename
    # only take the caseid that is in the icu_days_df
    if caseid not in icu_days_df["caseid"].values:
        continue
    df = pd.read_csv(filename)
    # add caseid to the dataframe
    df["caseid"] = caseid
    case_data[caseid] = df
    # drop first 40 rows and last 40 rows
    case_data[caseid] = case_data[caseid].iloc[300:-200]
    # drop every 2nd row
    case_data[caseid] = case_data[caseid].iloc[::2].reset_index(drop=True)


In [6]:
# Merge all data into a single dataframe
all_data = pd.concat(case_data.values(), ignore_index=True)

In [7]:


# Merge with icu_days_df
merged_data = pd.merge(all_data, icu_days_df, on='caseid')


# save the merged data
merged_data.to_csv("merged_data.csv", index=False)

# Select relevant columns
columns = [
    'SNUADC/ART', 'SNUADC/CVP', 'SNUADC/ECG_II', 'SNUADC/ECG_V5', 'SNUADC/FEM', 'SNUADC/PLETH',
    'Solar8000/ART_DBP', 'Solar8000/ART_MBP', 'Solar8000/ART_SBP', 'Solar8000/BT', 'Solar8000/CVP',
    'Solar8000/ETCO2', 'Solar8000/FEM_DBP', 'Solar8000/FEM_MBP', 'Solar8000/FEM_SBP', 'Solar8000/FEO2',
    'Solar8000/FIO2', 'Solar8000/GAS2_EXPIRED', 'Solar8000/GAS2_INSPIRED', 'Solar8000/HR',
    'Solar8000/INCO2', 'Solar8000/NIBP_DBP', 'Solar8000/NIBP_MBP', 'Solar8000/NIBP_SBP', 'Solar8000/PA_DBP',
    'Solar8000/PA_MBP', 'Solar8000/PA_SBP', 'Solar8000/PLETH_HR', 'Solar8000/PLETH_SPO2', 'Solar8000/RR',
    'Solar8000/RR_CO2', 'Solar8000/ST_AVF', 'Solar8000/ST_AVL', 'Solar8000/ST_AVR', 'Solar8000/ST_I',
    'Solar8000/ST_II', 'Solar8000/ST_III', 'Solar8000/ST_V5', 'Solar8000/VENT_COMPL', 'Solar8000/VENT_INSP_TM',
    'Solar8000/VENT_MAWP', 'Solar8000/VENT_MEAS_PEEP', 'Solar8000/VENT_MV', 'Solar8000/VENT_PIP',
    'Solar8000/VENT_PPLAT', 'Solar8000/VENT_RR', 'Solar8000/VENT_SET_FIO2', 'Solar8000/VENT_SET_PCP',
    'Solar8000/VENT_SET_TV', 'Solar8000/VENT_TV', 'Primus/AWP', 'Primus/CO2', 'Primus/COMPLIANCE',
    'Primus/ETCO2', 'Primus/EXP_DES', 'Primus/EXP_SEVO', 'Primus/FEN2O', 'Primus/FEO2', 'Primus/FIN2O',
    'Primus/FIO2', 'Primus/FLOW_AIR', 'Primus/FLOW_N2O', 'Primus/FLOW_O2', 'Primus/INCO2', 'Primus/INSP_DES',
    'Primus/INSP_SEVO', 'Primus/MAC', 'Primus/MAWP_MBAR', 'Primus/MV', 'Primus/PAMB_MBAR', 'Primus/PEEP_MBAR',
    'Primus/PIP_MBAR', 'Primus/PPLAT_MBAR', 'Primus/RR_CO2', 'Primus/SET_FIO2',
    'Primus/SET_FLOW_TRIG', 'Primus/SET_FRESH_FLOW', 'Primus/SET_INSP_PAUSE', 'Primus/SET_INSP_PRES',
    'Primus/SET_INSP_TM', 'Primus/SET_INTER_PEEP', 'Primus/SET_PIP', 'Primus/SET_RR_IPPV', 'Primus/SET_TV_L',
    'Primus/TV', 'Orchestra/AMD_RATE', 'Orchestra/AMD_VOL', 'Orchestra/DEX2_RATE',
    'Orchestra/DEX2_VOL', 'Orchestra/DEX4_RATE', 'Orchestra/DEX4_VOL', 'Orchestra/DOBU_RATE', 'Orchestra/DOBU_VOL',
    'Orchestra/DOPA_RATE', 'Orchestra/DOPA_VOL', 'Orchestra/DTZ_RATE', 'Orchestra/DTZ_VOL', 'Orchestra/EPI_RATE',
    'Orchestra/EPI_VOL', 'Orchestra/FUT_RATE', 'Orchestra/FUT_VOL', 'Orchestra/MRN_RATE', 'Orchestra/MRN_VOL',
    'Orchestra/NEPI_RATE', 'Orchestra/NEPI_VOL', 'Orchestra/NPS_RATE', 'Orchestra/NPS_VOL', 'Orchestra/NTG_RATE',
    'Orchestra/NTG_VOL', 'Orchestra/OXY_RATE', 'Orchestra/OXY_VOL', 'Orchestra/PGE1_RATE', 'Orchestra/PGE1_VOL',
    'Orchestra/PHEN_RATE', 'Orchestra/PHEN_VOL', 'Orchestra/PPF20_CE', 'Orchestra/PPF20_CP', 'Orchestra/PPF20_CT',
    'Orchestra/PPF20_RATE', 'Orchestra/PPF20_VOL', 'Orchestra/RFTN20_CE', 'Orchestra/RFTN20_CP', 'Orchestra/RFTN20_CT',
    'Orchestra/RFTN20_RATE', 'Orchestra/RFTN20_VOL', 'Orchestra/RFTN50_CE', 'Orchestra/RFTN50_CP', 'Orchestra/RFTN50_CT',
    'Orchestra/RFTN50_RATE', 'Orchestra/RFTN50_VOL', 'Orchestra/ROC_RATE', 'Orchestra/ROC_VOL', 'Orchestra/VASO_RATE',
    'Orchestra/VASO_VOL', 'Orchestra/VEC_RATE', 'Orchestra/VEC_VOL', 'BIS/BIS', 'BIS/EEG1_WAV', 'BIS/EEG2_WAV',
    'BIS/EMG', 'BIS/SEF', 'BIS/SQI', 'BIS/SR', 'BIS/TOTPOW', 'Invos/SCO2_L', 'Invos/SCO2_R', 'Vigileo/CI', 'Vigileo/CO',
    'Vigileo/SV', 'Vigileo/SVI', 'Vigileo/SVV', 'EV1000/ART_MBP', 'EV1000/CI', 'EV1000/CO', 'EV1000/CVP', 'EV1000/SV',
    'EV1000/SVI', 'EV1000/SVR', 'EV1000/SVRI', 'EV1000/SVV', 'Vigilance/BT_PA', 'Vigilance/CI', 'Vigilance/CO',
    'Vigilance/EDV', 'Vigilance/EDVI', 'Vigilance/ESV', 'Vigilance/ESVI', 'Vigilance/HR_AVG', 'Vigilance/RVEF',
    'Vigilance/SNR', 'Vigilance/SQI', 'Vigilance/SV', 'Vigilance/SVI', 'Vigilance/SVO2', 'CardioQ/ABP', 'CardioQ/FLOW',
    'CardioQ/CI', 'CardioQ/CO', 'CardioQ/FTc', 'CardioQ/FTp', 'CardioQ/HR', 'CardioQ/MA', 'CardioQ/MD', 'CardioQ/PV',
    'CardioQ/SD', 'CardioQ/SV', 'CardioQ/SVI', 'FMS/FLOW_RATE', 'FMS/INPUT_AMB_TEMP', 'FMS/INPUT_TEMP', 'FMS/OUTPUT_AMB_TEMP',
    'FMS/OUTPUT_TEMP', 'FMS/PRESSURE', 'FMS/TOTAL_VOL', 'icu_days', 'caseid'
]


existing_columns = [col for col in columns if col in merged_data.columns]
missing_columns = [col for col in columns if col not in merged_data.columns]

print(f"Missing columns: {missing_columns}")
print(f"Existing columns: {existing_columns}")
# save existing columns as npy
np.save("existing_columns.npy", existing_columns)


data = merged_data[existing_columns]

# Handle missing values
data = data.fillna(method='ffill').fillna(method='bfill')

# Save the data
data.to_csv("preprocessed_data.csv", index=False)



Missing columns: ['SNUADC/ART', 'SNUADC/CVP', 'SNUADC/ECG_V5', 'SNUADC/FEM', 'Solar8000/ART_DBP', 'Solar8000/ART_MBP', 'Solar8000/ART_SBP', 'Solar8000/BT', 'Solar8000/CVP', 'Solar8000/FEM_DBP', 'Solar8000/FEM_MBP', 'Solar8000/FEM_SBP', 'Solar8000/GAS2_EXPIRED', 'Solar8000/GAS2_INSPIRED', 'Solar8000/NIBP_DBP', 'Solar8000/NIBP_MBP', 'Solar8000/NIBP_SBP', 'Solar8000/PA_DBP', 'Solar8000/PA_MBP', 'Solar8000/PA_SBP', 'Solar8000/RR', 'Solar8000/ST_AVF', 'Solar8000/ST_AVL', 'Solar8000/ST_AVR', 'Solar8000/ST_I', 'Solar8000/ST_II', 'Solar8000/ST_III', 'Solar8000/ST_V5', 'Solar8000/VENT_COMPL', 'Solar8000/VENT_INSP_TM', 'Solar8000/VENT_MEAS_PEEP', 'Solar8000/VENT_PIP', 'Solar8000/VENT_PPLAT', 'Solar8000/VENT_RR', 'Solar8000/VENT_SET_FIO2', 'Solar8000/VENT_SET_PCP', 'Solar8000/VENT_SET_TV', 'Solar8000/VENT_TV', 'Primus/EXP_DES', 'Primus/EXP_SEVO', 'Primus/FLOW_AIR', 'Primus/FLOW_N2O', 'Primus/FLOW_O2', 'Primus/INSP_DES', 'Primus/INSP_SEVO', 'Primus/PEEP_MBAR', 'Primus/PPLAT_MBAR', 'Primus/SET_FLOW

/var/folders/4x/bz2824s146n10c63_0mz1y840000gn/T/ipykernel_88353/1246674356.py:61: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data = data.fillna(method='ffill').fillna(method='bfill')


In [5]:
print(f'X_train shape: {X_train.shape}')  # Expected shape: (number of samples, sequence_length, number of features)
print(f'X_test shape: {X_test.shape}')
print(f'y_train shape: {y_train.shape}')  # Expected shape: (number of samples,)
print(f'y_test shape: {y_test.shape}')

X_train shape: (943487, 100, 31)
X_test shape: (235872, 100, 31)
y_train shape: (943487,)
y_test shape: (235872,)


In [8]:
# import model icu_model.h5
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
import pickle

# import icu_model.h5 and history_dict
existing_columns = np.load("existing_columns.npy")

model = load_model('my_model.keras')

history = pickle.load(open('history_dict', "rb"))
# Load the sequences and targets
X = np.load("X.npy")
y = np.load("y.npy")

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [35]:
# Check for NaN values in the sequences and targets
print(f"NaN values in X: {np.isnan(X).sum()}")
print(f"NaN values in y: {np.isnan(y).sum()}")

NaN values in X: 0
NaN values in y: 0


In [16]:
from sklearn.metrics import mean_squared_error
import numpy as np

def permutation_feature_importance(model, X, y, metric=mean_squared_error, n_repeats=10):
    baseline = metric(y, model.predict(X))
    importances = np.zeros(X.shape[2])

    for i in range(X.shape[2]):
        score_drops = []
        for _ in range(n_repeats):
            X_permuted = X.copy()
            np.random.shuffle(X_permuted[:, :, i])
            permuted_score = metric(y, model.predict(X_permuted))
            score_drops.append(permuted_score - baseline)
        importances[i] = np.mean(score_drops)
    
    return importances

# Calculate feature importances
importances = permutation_feature_importance(model, X_test, y_test)
sorted_indices = np.argsort(importances)[::-1]

# Print feature importances
for idx in sorted_indices:
    print(f"Feature: {existing_columns[idx]}, Importance: {importances[idx]}")

2146/2146 ━━━━━━━━━━━━━━━━━━━━ 27s 12ms/step
2146/2146 ━━━━━━━━━━━━━━━━━━━━ 26s 12ms/step
2146/2146 ━━━━━━━━━━━━━━━━━━━━ 26s 12ms/step
2146/2146 ━━━━━━━━━━━━━━━━━━━━ 27s 12ms/step
2146/2146 ━━━━━━━━━━━━━━━━━━━━ 26s 12ms/step
2146/2146 ━━━━━━━━━━━━━━━━━━━━ 26s 12ms/step
2146/2146 ━━━━━━━━━━━━━━━━━━━━ 26s 12ms/step
2146/2146 ━━━━━━━━━━━━━━━━━━━━ 26s 12ms/step
2146/2146 ━━━━━━━━━━━━━━━━━━━━ 26s 12ms/step
2146/2146 ━━━━━━━━━━━━━━━━━━━━ 26s 12ms/step
2146/2146 ━━━━━━━━━━━━━━━━━━━━ 26s 12ms/step
2146/2146 ━━━━━━━━━━━━━━━━━━━━ 26s 12ms/step
2146/2146 ━━━━━━━━━━━━━━━━━━━━ 26s 12ms/step
2146/2146 ━━━━━━━━━━━━━━━━━━━━ 27s 12ms/step
2146/2146 ━━━━━━━━━━━━━━━━━━━━ 27s 13ms/step
2146/2146 ━━━━━━━━━━━━━━━━━━━━ 27s 13ms/step
2146/2146 ━━━━━━━━━━━━━━━━━━━━ 27s 13ms/step
2146/2146 ━━━━━━━━━━━━━━━━━━━━ 27s 13ms/step
2146/2146 ━━━━━━━━━━━━━━━━━━━━ 27s 13ms/step
2146/2146 ━━━━━━━━━━━━━━━━━━━━ 27s 13ms/step
2146/2146 ━━━━━━━━━━━━━━━━━━━━ 27s 12ms/step
2146/2146 ━━━━━━━━━━━━━━━━━━━━ 27s 13ms/step
2146/2146 

In [16]:
# import shap

# # Create a SHAP explainer
# explainer = shap.DeepExplainer(model, X_train[:100])  # Use a subset of the training data to fit the explainer

# # Calculate SHAP values
# shap_values = explainer.shap_values(X_test[:100])  # Use a subset of the test data for the explanation

# # Plot summary plot
# shap.summary_plot(shap_values, X_test[:100], feature_names=existing_columns[:-2])

In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation loss values
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.show()

y_pred = model.predict(X_test)

# Plot the actual vs predicted values
plt.figure(figsize=(10, 6))
plt.plot(y_test, label='Actual ICU Days')
plt.plot(y_pred, label='Predicted ICU Days')
plt.title('Actual vs Predicted ICU Days')
plt.xlabel('Sample')
plt.ylabel('ICU Days')
plt.legend(loc='upper right')
plt.show()

In [14]:
# get case with icu days = 2
case = case_df[case_df['icu_days'] == 2].iloc[1]
print(case)

caseid             44
subjectid        5052
casestart           0
caseend         14614
anestart        -1341
                ...  
intraop_vecu        0
intraop_eph        10
intraop_phe         0
intraop_epi         0
intraop_ca          0
Name: 43, Length: 74, dtype: object


In [15]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np

caseid = 44

# Load the CSV file for a single patient
patient_data = pd.read_csv(f"vital_data/case_{caseid}.csv")
# drop first and alst 40 rows
patient_data = patient_data.iloc[40:-40]

patient_data = patient_data.fillna(method='ffill').fillna(method='bfill')
# compare columns with existing_columns
#missing_columns = [col for col in patient_data.columns if col not in existing_columns]
#print(f"Missing columns: {missing_columns}")
# drop time
patient_data = patient_data.drop(columns=["time"])


#get the icu days from icu_days_df for the caseid
icu_days = case_df[case_df["caseid"] == caseid]["icu_days"].values[0]
print("Actual ICU Days:", icu_days)
# Standardize the data
scaler = StandardScaler()
patient_data_scaled = scaler.fit_transform(patient_data)

# Convert to DataFrame to maintain column names
patient_data_scaled = pd.DataFrame(patient_data_scaled, columns=existing_columns[:-2])

print("Patient data loaded and standardized.")

# Create sequences function for a single patient
def create_patient_sequence(df, sequence_length):
    sequences = []
    if len(df) >= sequence_length:
        for i in range(len(df) - sequence_length):
            seq = df.iloc[i:i+sequence_length][existing_columns[:-2]].values
            sequences.append(seq)
    return np.array(sequences)

# Set the sequence length
sequence_length = 1000  # Use the same value as used during training

# Create the sequence
X_patient = create_patient_sequence(patient_data_scaled, sequence_length)

print(f"Patient sequence created: X_patient shape: {X_patient.shape}")

import tensorflow as tf

# Ensure that the sequence length and number of features match the model's expected input shape
if X_patient.shape[1:] == (sequence_length, len(existing_columns) - 2):
    # Make predictions
    y_pred_patient = model.predict(X_patient)
    print(y_pred_patient)
    # Average the predictions if multiple sequences were created
    predicted_icu_days = np.mean(y_pred_patient)
    print(f"Predicted ICU days for the patient: {predicted_icu_days}")
else:
    print("Error: The input sequence does not match the expected shape for the model.")

/var/folders/4x/bz2824s146n10c63_0mz1y840000gn/T/ipykernel_88353/2194713673.py:12: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  patient_data = patient_data.fillna(method='ffill').fillna(method='bfill')


Actual ICU Days: 2
Patient data loaded and standardized.
Patient sequence created: X_patient shape: (13534, 1000, 30)
423/423 ━━━━━━━━━━━━━━━━━━━━ 33s 77ms/step
[[2.2535954 ]
 [2.2510667 ]
 [2.2384748 ]
 ...
 [0.14529228]
 [0.19171143]
 [0.27528572]]
Predicted ICU days for the patient: 2.0839741230010986


In [17]:
from timeseries_predict import predict_icu_days_from_data

caseid = 44

# Load the CSV file for a single patient
patient_data = pd.read_csv(f"vital_data/case_{caseid}.csv")
# get the icu days from icu_days_df for the caseid
icu_days = case_df[case_df["caseid"] == caseid]["icu_days"].values[0]
print("Actual ICU Days:", icu_days)

predict_icu_days_from_data(patient_data=patient_data)

Actual ICU Days: 2
Patient data loaded and standardized.


/Users/michael/Library/Mobile Documents/com~apple~CloudDocs/TUM/EI TUM Master/Term 3/Cacom/CAoCM_Vital/timeseries_predict.py:15: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  patient_data = patient_data.fillna(method="ffill").fillna(method="bfill")


Patient sequence created: X_patient shape: (13374, 1000, 30)
418/418 ━━━━━━━━━━━━━━━━━━━━ 32s 77ms/step
[[3.6705608 ]
 [3.6497812 ]
 [3.6333656 ]
 ...
 [0.19579506]
 [0.24669933]
 [0.33993053]]


2.0874212